In [1]:
import sys
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from pathlib import Path
import myDataloader as db
import matplotlib.pyplot as plt

Loading Dataset....
...................................................................... Completed
Number of data: 735
Batch Size: 1024
Number of Batches: 1
--------------------------------------------------------------------------------


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 0
torch.manual_seed(SEED)
np.random.seed(SEED)

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.numLayers = 1
        self.input_dim = 1
        self.output_dim = self.input_dim
        self.hidden_size = 128
        self.batchSize = db.batchSize  # or numData when numData < batchSize
        self.numData = db.numSequences
        if(self.numData <= self.batchSize):
            self.batchSize = self.numData

        self.lstm1 = nn.LSTM(self.input_dim, self.hidden_size)
        self.linear = nn.Linear(self.hidden_size, self.output_dim)

        h0 = torch.zeros(self.numLayers, self.numData, self.hidden_size).to(device)
        self.h0 = nn.Parameter(h0, requires_grad=True)


    def forward(self, input, label):
        seqLength = input.shape[0]
        numDataInBatch = input.shape[1]
        for i in range(len(label)):
            idxSeq = label[i][0]
            temp = self.h0[:,idxSeq,:].view(1,1,-1)
            if(i == 0):
                h0 = temp
            else:
                h0 = torch.cat((h0, temp), 1)

        for t in range(seqLength):
            if(t ==0):
                h_t = h0
                c_t = torch.zeros(self.numLayers, numDataInBatch, self.hidden_size).to(device)
                input_t = torch.zeros(1, numDataInBatch, self.input_dim).to(device)  # firstDim = time = 1
            else:
                input_t = output

            output, (h_t, c_t) = self.lstm1(input_t, (h_t, c_t))

            output = self.linear(h_t)

            if (t == 0):
                outputs = output
            else:
                outputs = torch.cat((outputs,output),0)

        return outputs

In [4]:
checkpoint = torch.load('./result/model_best.tar', map_location=torch.device('cpu'))
learningRate = 0.001
numEpoch = 100000

model = Sequence().to(device)
optimizer = Adam(params=model.parameters(), lr=learningRate)
criterion = nn.MSELoss()

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import glob, os

class myDataset(Dataset):
    def __init__(self, data):
        self.data_input, self.data_label = data
    def __getitem__(self, index):
        return self.data_input[index], self.data_label[index]
    def __len__(self):
        return len(self.data_input)


In [48]:

DATA_TR = []
DATA_INPUT = []
DATA_LABEL = []

# ================================================================================================
# These variables are used in other python files as well
batchSize = 1024
numSequences = 0
seqLenth = 241 # Length of the sequence.
# ================================================================================================

print('=' * 80)
print('Loading Dataset....')

txtFiles = glob.glob("./data/2019-01-01_2019-12-31/*KS*.txt")
txtFiles = sorted(txtFiles)
numFiles = len(txtFiles)

maxLength  = 0

for idx in range(numFiles):
    posFilename = txtFiles[idx]
    data = torch.Tensor(np.loadtxt(posFilename))
    # print(type(data))
    # print(data.shape)

    scalingFactor = data[0]
    data = data / scalingFactor
    data = data - 1.0 

    DATA_INPUT.append(data)
    DATA_LABEL.append([idx])
    numSequences += 1




Loading Dataset....


In [49]:
type(DATA_INPUT[0])

torch.Tensor

In [43]:
DATA_INPUT[0][0]

tensor(0.)

In [27]:
db.data_loader

AttributeError: 'DataLoader' object has no attribute 'size'

In [35]:
path = "./data/2019-01-01_2019-12-31/000020.KS.txt"
data = torch.Tensor(np.loadtxt(posFilename)).numpy().tolist()
avg = data[0]
sum1 = sum(data[1:])
print(avg)
print(sum1/240.0)

2397.0
3123.55
